In [ ]:
import boto3
from sagemaker.s3 import S3Uploader
from sagemaker.session import Session
import dotenv

from datasets import load_from_disk
from transformers import AutoTokenizer
import pandas as pd

dotenv.load_dotenv()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/vini/.config/sagemaker/config.yaml


/home/vini/Documents/sagemaker_text_classifier/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

# Preprocessing data

## 1. Tokenizing dataset

In [ ]:
# Load IMDb dataset
dataset = load_from_disk("data/raw/imdb")
train_data, test_data = dataset["train"], dataset["test"]

# Initialize tokenizer (using DistilBERT for consistency)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenization function
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)

# Tokenize datasets
train_dataset = train_data.map(tokenize, batched=True)
test_dataset = test_data.map(tokenize, batched=True)

# Save processed datasets to disk
train_dataset.save_to_disk("data/preprocessed/train_processed")
test_dataset.save_to_disk("data/preprocessed/test_processed")

## 2. Upload tokenized data to AWS

In [7]:
# aws setup

# Manually set AWS credentials and region (replace placeholders)
aws_access_key_id = dotenv.get_key(dotenv.find_dotenv(), "ACCESS_KEY")  
aws_secret_access_key = dotenv.get_key(dotenv.find_dotenv(), "SECRET_KEY") 
region_name = "us-east-1"                  # Or your preferred SageMaker region (e.g., "eu-west-1")

# Configure the default session
boto3.setup_default_session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name
)

# Verify the region is set
print("Current AWS Region:", boto3.Session().region_name)

Current AWS Region: None


In [ ]:
# Upload data to S3 bucket
bucket_name = "sagemaker-us-east-1-339712837044"
# S3Uploader.upload("data/preprocessed/train_processed", f"s3://{bucket_name}/text-classifier/train/")
# S3Uploader.upload("data/preprocessed/test_processed", f"s3://{bucket_name}/text-classifier/test/")

's3://sagemaker-us-east-1-339712837044/text-classifier/test/'